## Understanding Perplexity in Language Models

To understand and compute perplexity, a key evaluation metric for language models, and analyze how it reflects the quality of text predictions.

**What is Perplexity?**
Perplexity measures the uncertainty of a language model in predicting a sequence of words. It indicates how "perplexed" the model is by the text.

- Low perplexity: The model predicts the sequence with high confidence.
- High perplexity: The model struggles to predict the sequence, indicating poor performance.






In [1]:
import tensorflow as tf
from transformers import TFAutoModelForCausalLM, AutoTokenizer

def calculate_perplexity(text, model_name='gpt2'):
    """
    Calculates the perplexity of the given text using a GPT-2 model in TensorFlow.
    
    Args:
        text (str): Input text.
        model_name (str): Name of the Hugging Face model (default: 'gpt2').
    
    Returns:
        float: Perplexity score.
    """
    
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = TFAutoModelForCausalLM.from_pretrained(model_name)
    
    # Tokenize the input text
    tokens = tokenizer.encode(text, return_tensors='tf')

    # Log the tokenized input
    print(f"\nOriginal Text: {text}")
    print(f"Tokenized Input: {tokens}")

    # Calculate loss and perplexity
    outputs = model(tokens, labels=tokens)
    loss = outputs.loss
    perplexity = tf.exp(loss)

    print(f"Loss: {loss.numpy()}")
    return perplexity.numpy()

# Compare perplexity of different examples
texts = [
    "The quick brown fox jumps over the lazy dog.",  # Grammatically correct and meaningful
    "Quick the brown fox over lazy jumps dog the.",  # Grammatically incorrect and jumbled
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit.",  # Latin placeholder text
    "Random gibberish xzq mfnweor pasd."  # Completely random gibberish
]

print("\n--- Perplexity Comparison ---")
for text in texts:
    perplexity = calculate_perplexity(text)
    print(f"Perplexity: {perplexity}")



--- Perplexity Comparison ---


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.



Original Text: The quick brown fox jumps over the lazy dog.
Tokenized Input: [[  464  2068  7586 21831 18045   625   262 16931  3290    13]]
Loss: [5.0905147]
Perplexity: [162.47346]


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.



Original Text: Quick the brown fox over lazy jumps dog the.
Tokenized Input: [[21063   262  7586 21831   625 16931 18045  3290   262    13]]
Loss: [8.565364]
Perplexity: [5246.7485]


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.



Original Text: Lorem ipsum dolor sit amet, consectetur adipiscing elit.
Tokenized Input: [[   43 29625   220  2419   388   288 45621  1650   716   316    11   369
   8831   316   333 31659   271  2259  1288   270    13]]
Loss: [0.9613916]
Perplexity: [2.6153336]


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.



Original Text: Random gibberish xzq mfnweor pasd.
Tokenized Input: [[29531 46795   527   680  2124    89    80   285 22184   732   273 38836
     67    13]]
Loss: [6.982953]
Perplexity: [1078.0974]


Lower perplexity for natural text (first example) indicates the model is confident in predicting the sequence.
Higher perplexity for random gibberish reflects the model's struggle to make predictions.

## Imports and Libraries

In [2]:
pip install --upgrade tensorflow-datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from collections import Counter

### Loading dataset

We will use the IMDb movie reviews dataset, which is a collection of movie reviews along with sentiment labels. We will focus on the text data and ignore the labels for this task.

**as_supervised=True** allows us to retrieve the data in a format where the input data is paired with its label (although we won't use the labels in this case).


In [4]:
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)


## Preprocess the Text Data

The IMDb dataset contains raw text data that we need to preprocess. We will tokenize the text (split the text into words) and convert them into trigrams (3 consecutive words).

Here, the tokenize() function converts the text from a byte string to a regular Python string and then splits it into individual words.

In [5]:
def tokenize(text):
    return text.numpy().decode('utf-8').split()

def extract_trigrams(text):
    words = tokenize(text)
    trigrams = [(words[i], words[i + 1], words[i+2]) for i in range(len(words) - 2)]
    return trigrams


### Limit Data Size for Training and Testing
To keep the experiment manageable, we will limit the training and test data to a smaller number of samples (500 for training and 100 for testing):

In [6]:
train_data = dataset['train'].map(lambda x, y: x)
test_data = dataset['test'].map(lambda x, y: x)

train_texts = list(train_data.take(500))  # Limit to 500 training samples
test_texts = list(test_data.take(100))    # Limit to 100 test samples


2024-11-21 17:49:31.472832: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-11-21 17:49:31.495157: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### Build Vocabulary and Convert Words to Indices 

In [7]:
train_trigrams = []
for text in train_texts:
    train_trigrams.extend(extract_trigrams(text))

test_trigrams = []
for text in test_texts:
    test_trigrams.extend(extract_trigrams(text))

train_words = [w for trigram in train_trigrams for w in trigram]
test_words = [w for trigram in test_trigrams for w in trigram]

vocab = list(set(train_words))
vocab_size = len(vocab)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
word_to_idx["<UNK>"] = vocab_size  # Add a special token for unknown words
idx_to_word = {idx: word for word, idx in word_to_idx.items()}


## Convert Words to Indices

Every word in the training and test sets is replaced by its corresponding index in the vocabulary.

In [8]:
def word_to_index(word):
    return word_to_idx.get(word, word_to_idx["<UNK>"])  # Use <UNK> for unknown words

train_sequences = [word_to_index(word) for word in train_words]
test_sequences = [word_to_index(word) for word in test_words]


### Prepare Data for the LSTM Model

From the sequences of word indices, we need to prepare the data for the LSTM model. Specifically, we create input-output pairs where the input is a sequence of words, and the output is the next word in the sequence.

In [9]:
def create_input_output(sequences, sequence_length=2):
    X, y = [], []
    for i in range(len(sequences) - sequence_length):
        X.append(sequences[i:i + sequence_length - 1])
        y.append(sequences[i + sequence_length - 1])
    return np.array(X), np.array(y)

X_train, y_train = create_input_output(train_sequences)
X_test, y_test = create_input_output(test_sequences)


### Build the LSTM Model

The model will have the following layers:

- Embedding Layer: Converts word indices into dense word embeddings.
- LSTM Layer: Processes the sequence of words to capture temporal dependencies.
- Dense Layer: Outputs the probability distribution over all possible next words.

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size + 1, output_dim=128, input_length=X_train.shape[1]),  # +1 for <UNK>
    tf.keras.layers.LSTM(128, return_sequences=False),
    tf.keras.layers.Dense(vocab_size + 1, activation='softmax')  # +1 for <UNK>
])


### Compile and Train the Model

In [11]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/3
5093/5093 [==============================] - 65s 13ms/step - loss: 7.4481 - accuracy: 0.0667 - val_loss: 7.5404 - val_accuracy: 0.0773
Epoch 2/3
5093/5093 [==============================] - 59s 12ms/step - loss: 6.8613 - accuracy: 0.0994 - val_loss: 7.7460 - val_accuracy: 0.0886
Epoch 3/3
5093/5093 [==============================] - 58s 11ms/step - loss: 6.4854 - accuracy: 0.1209 - val_loss: 7.9276 - val_accuracy: 0.0925


### Calculate Perplexity

In [12]:
def calculate_perplexity(model, X, y):
    with tf.device('/GPU:0'):
        predictions = model.predict(X)
        log_prob_sum = 0
        N = len(y)
        
        for i in range(N):
            prob = predictions[i, y[i]]
            log_prob_sum += np.log(prob + 1e-10)  # Smoothing to avoid log(0)
        
        perplexity = np.exp(-log_prob_sum / N)
        return perplexity

perplexity = calculate_perplexity(model, X_test, y_test)
print(f'Perplexity: {perplexity}')


1896/1896 [==============================] - 4s 2ms/step
Perplexity: 2765.4765572578326


### Strategies to improve the score

- Limited Context:

    A trigram model relies on only two preceding words to predict the next word. This is often insufficient for capturing long-range dependencies in natural language.
    Many words in a sentence depend on context from earlier words or the entire sentence, not just the last two words.

- Small or Insufficient Data:

    Language models require large amounts of data to estimate probabilities accurately, especially for trigrams where combinations of three words must be seen during training.
    Sparse Data Problem:

    Trigram models suffer from sparsity, as many possible word combinations may not appear in the training data. This makes it difficult for the model to generalize.
    Vocabulary Size:

    A large vocabulary size increases the model's difficulty in accurately estimating probabilities for rare or unseen words, leading to higher perplexity.